In [1]:
import os 
import math 
import random 
import shutil

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

import tensorflow as tf
import keras


In [6]:
BASE_DIR = "C:/Sowmya/Personal/TENSORFLOW/Tensorflow_stuffs/lego/star-wars-images/"
names = ["YODA", "LUKE SKYWALKER", "R2-D2", "MACE WINDU", "GENERAL GRIEVOUS"]

tf.random.set_seed(1)

In [7]:
if not os.path.isdir(BASE_DIR + 'train/'):
    for name in names:
        os.makedirs(BASE_DIR + 'train/' + name)
        os.makedirs(BASE_DIR + 'val/' + name)
        os.makedirs(BASE_DIR + 'test/' + name)

In [8]:
# Moce the image files
orig_folders = ["0001/", "0002/", "0003/", "0004/", "0005/"]
for folder_idx, folder in enumerate(orig_folders):
    files = os.listdir(BASE_DIR + folder)
    number_of_images = len([name for name in files])
    n_train = int((number_of_images * 0.6) + 0.5)
    n_valid = int((number_of_images*0.25) + 0.5)
    n_test = number_of_images - n_train - n_valid
    print(number_of_images, n_train, n_valid, n_test)
    for idx, file in enumerate(files):
        file_name = BASE_DIR + folder + file
        if idx < n_train:
            shutil.move(file_name, BASE_DIR + "train/" + names[folder_idx])
        elif idx < n_train + n_valid:
            shutil.move(file_name, BASE_DIR + "val/" + names[folder_idx])
        else:
            shutil.move(file_name, BASE_DIR + "test/" + names[folder_idx])

12 7 3 2
12 7 3 2
12 7 3 2
13 8 3 2
14 8 4 2


In [9]:
# Generate batches of tensor image data with optional real-time data augmentation.

# preprocessing_function
# rescale = 1./255 -> [0, 1]

train_gen= keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

AttributeError: module 'keras.api.preprocessing.image' has no attribute 'ImageDataGenerator'